<a href="https://colab.research.google.com/github/Ignacio-Ibarra/NLP-Disasters/blob/master/Real_or_Not%3F_NLP_with_Disaster_Tweets_Informe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real or Not? NLP with Disaster Tweets**

### **Autores**: Alejandra Toscano - Diego Sanchez - Erik Cirilo - Ignacio Ibarra

## **Introducción** 

Esta es un trabajo práctico realizado para la materia 7506 - Organización de Datos de la Facultad de Ingniería de la Universidad de Buenos Aires. Se trata de una competencia lanzada en la plataforma Kaggle (ver [aquí](https://www.kaggle.com/c/nlp-getting-started)), la cual consiste en predecir si un tweet nos está hablando de una desastre o no, implementando técnicas de _natural language processing_ (NLP).   

Para ello contamos con un _dataset_ de entrenamiento y otro para testear nuestro modelo de _machine learning._ 

Este trabajo consta de dos entregas: 

* **Primera parte: Análisis Exploratorio de los Datos**

>El principal objetivo de esta parte es hallar algunos _insights_ mediante visualizaciones de las distintas relaciones entre las variables. Este trabajo se realizará sobre el _dataset_ de entrenamiento. Las variables presentes en el mismo son: 

  >- `id` : una identificación única de cada una de las observaciones
  
  >-  `keyword`: es una palabra que el usuario de Twitter puede cargar al momento de postear un tweet con el objetivo de asociar el texto a dicha palabra o concepto. Podría ser totalmente arbitraria la elección del _keyword_

  >- `location`: el usuario también puede cargar la locación desde dónde realiza el posteo. También es arbitraria dicha elección porque no se carga desde la locación del GPS. 

  >- `text`: es el texto del tweet. 

  >- `target`: es una variable binaria que toma valor 1 cuando el tweet corresponde con un desastre y 0 cuando no. 

* **Segunda parte: Modelo de machine learning basado en NLP** 

>El objetivo de esta segunda parte es desarrollar un modelo que luego de ser probado en la base de entrenamiento podamos utilizarlo para predecir la variable target. 

## **Pre-procesamiento**



### **Limpieza del dataset** 



#### **Limpieza variable `text`**

Para trabajar con más claridad con esta variable hemos eliminado las url, los _hashtags_, las _mentions_ del texto de cada tweet, esto nos permitirá luego hacer un mejor análisis sobre los "duplicados". 

#### **Registros duplicados**

En primer lugar comprobamos que dado que el id es una clave única no hay filas duplicadas. 

Sin embargo, lo que cuenta es definir también un criterio para contabilizar valores como 'duplicados'. Para ello hemos redefinido la variable `text` como una variable que alberga sólo oraciones (sin url, _hashtags_, _mentions_)  y hemos considerado los repetidos sin tener en cuenta el `id`. No se consideraron diferencias en la puntuación.  
 
De esta manera comprobamos que había muchos textos repetidos y que además tenían valores de `target` contradictorios. Para un mismo texto había dos o más filas que tenían distinto valor de `target`. Como se ve en la siguiente tabla. 

texto|count|number_of_uniques
---|:---:|:---:|		
Enugu Government to demolish illegal structures at International Conference Centre	|3|	2
.POTUS is a strategy for refugees; IDP Internally displaced people; horror; etc.	|5|	2
Why are you deluged with low self-image? Take the quiz:|	2	|2
Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) -|	5	|2
Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...	|2	|2
RT NotExplained: The only known image of infamous hijacker D.B. Cooper.	|2	|2
FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps via	|4	|2
that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time|4|	2
slips into loss after unsafe and hazardous for|	3	|2
TY for the follow Go To BRUTALLY ABUSED+DESOLATE&amp;LOST + HER LOVELY MUM DIES..Is it Murder?|	2|	2
The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'|	6	|2
To fight bioterrorism sir.|	4	|2
Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs: An angry Internally Displaced wom...|	3|	2
describes piling up thinking it would last as the description of the people of in Surah Humaza.	|3|	2
Sassy city girl country hunk stranded in Smoky Mountain snowstorm|	3|	2

Los registros contradictorios fueron eliminados, pasando de tener 7613 a 7368 filas. 

#### **Tratamiento _null values_**

Por otro lado, hemos reemplazado los _null values_ de las columnas `keyword` y `location` por 'no_keyword' y 'no_location' respectivamente, para no perder ni filas ni columnas. 

### **Nuevas variables creadas** (corregir)

Hemos creado nuevas variables a partir de la variable `text`: 

>-`tweet_len`: cantidad de caracteres que posee cada tweet. 

>-`list_words`: hemos obtenido una lista de _strings_ para cada tweet. Y de esa variable hemos deducido otras como: 

>>-`words_count`: cantidad de palabras de cada tweet. 

>>>-`avg_characters_per_word`: es el cociente entre `words_count` y `tweet_len`

>>-`hashtag`: hemos obtenido los hashtags de cada tweet.

>>-`words_freq`: calculamos la frecuencia que tiene cada palabra. 

Para más información ver [aquí](poner link al repo)  

## **Proporción de Target Real y No Real**

La variable `target` está distribuida de la siguiente forma en el _dataset_ de entrenamiento. 

![Bar Chart - Cantidad de Tweets por Target](https://drive.google.com/uc?id=1xIAFSh3PyCGB1Z0pRFbykBULTl8IyJFH)

![Pie Chart - Porcentaje de Tweets por Target](https://drive.google.com/uc?id=1r7NAc0aH19QOXHxXlwMrrMaO3Z2cl1ng)


### **Analizando `keyword`**

La palabra clave es cargada por el usuario de Twitter al momento del post. Sirve para relacionar un tweet con un concepto y facilita que el motor de búsqueda de Twitter devuelva dicho post si alguien realiza una búsqueda utilizando dicha palabra clave. 

Si bien la elección del 'keyword' por parte del usuario puede guardar algún tipo de relación con el texto del tweet, no se puede asegurar que eso suceda ya que dicha elección es a _priori_ arbitraria. 

La variable en análisis puede presentarse como una única palabra o como una conjunción de más de una. 

Los _unique values_ son el 2.92% del total de registros de dicha variable. En la siguiente tabla podemos ver las palabras que más se repiten y las que menos, sin tomar en cuenta el valor 'no_keyword'.  

keyword|Frecuencia absoluta
---|---|
fatalities |              45
deluge      |             42
armageddon   |            42
harm          |           41
..|                         ..
forest%20fire     |       19
epicentre     |           12
threat        |           11
inundation    |           10
radiation%20emergency |    9

A su vez, el valor 'no_keyword' aparece 61 veces en la muestra, pero a los fines de saber cuáles son las palabras o conceptos que los tweets están volcando, el valor 'no_keyword' no aporta información valiosa. 

#### **`keyword` por `target`**

Agrupando por `keyword` obtenemos la proporción de registros que tienen un valor de `target` igual a 1 y calculamos el porcentaje sobre el total de registros para ese valor de `keyword`. De las 222 keywords hemos seleccionado las 25 con porcentaje más alto y las 25 con porcentaje más bajo. 

![Horizontal Bar Chart: Top 25 Keywords Frecuentes por Target](https://drive.google.com/uc?id=1rl4_gsGLuY1al6EJcH-oo-Q7Yr2ANN4e)

En la figura de la izquierda, se observan keywords referentes a desastres naturales (_drought, forest fire, violent storm) o edilicios (bridge collapse, buildings on fire_). También se encuentran tragedias mortales (_mass murder, airplane accident, nuclear disaster, suicide bombing_). Otros keywords incluyen lenguaje de noticias (_casualties, evacuated, rescuers_).

En la figura de la derecha, su mayoría se trata de palabras que servirían para describir tragedias pero que a su vez pueden usarse en muchos otros contextos. Un ejemplo de esto es el keyword "body bag", con un porcentaje de targets Reales menor al 5%. El término por si solo sonaría alarmante, pero veamos las acepciones de la palabra:

1.   Bolsa para guardar cadáveres
2.   Una clase de cartera/mochila
3.   Según *Urban Dictionary*, "To boddybag an oponent" es asegurar la victoria sobre el oponente (algo así como "kick ass"), es un slang muy utilizado en las batallas de rap.
4.   Otra acepción de *Urban Dictionary*, "Bodybagging" es robarle un chiste a otro comediante y hacerlo pasar como propio. 

#### **Frecuencia de Frecuencias** 

Por otro lado es llamativo observar la frecuencia de las frecuencias, es decir, cuántas veces se repiten ciertas frecuencias. La mayor parte de las `keyword` se repiten entre 30 y 40 veces. 

![Frecuencia de Frecuencias](https://drive.google.com/uc?id=1IpUAmYu1HgDe7qnnm84sBnMKzvaf6CJ_)

El 88,57% de los registros posee un valor de `keyword` que se encuentra dentro del grupo de las `keyword` que se repiten entre 30 y 40 veces. De las 222 keywords, 192 se repiten entre 30 y 40 veces. 

Observamos que no hay relación entre la cantidad de apariciones de un `keyword` y el porcentage de veracidad de dicho keyword, calculado éste como la media del `target` para cada keyword. 

![Sctatter plot: Relación entre el porcentaje de veracidad y frecuencia del `keyword` ](https://drive.google.com/uc?id=13CcSoLQG1Hwu4ToR9vZcj5x3eWAab9g5)

Lo mismo muestra la matriz de correlaciones. 

corr|percentage|	count
---|---|---
percentage|	1.00|	0.08
count	|0.08|	1.00


